# Taxi Demand Forecasting: Algorithm Comparisons (2025 Data)

**Workflow:**
1. Synthesize new taxi demand and weather datasets for Jan-May 2025 (shifting only time columns, preserving all other data).
2. Preprocess the new data using the same logic as previous notebooks.
3. Train and compare 7 modern algorithms (including boosting and balancing techniques) for demand forecasting.
4. Evaluate and compare model performance, aiming for 90-93% accuracy without overfitting.

> **Note:** This notebook does not modify or overwrite any existing notebooks or data files.


In [2]:
import os
import pandas as pd
from datetime import datetime, timedelta
import shutil

# --- Parameters ---
ORIG_ZONE_FILES = [
    'data/2017-01_1H_zone.csv',
    'data/2017-02_1H_zone.csv',
    'data/2017-03_1H_zone.csv',
    'data/2017-04_1H_zone.csv',
    'data/2017-05_1H_zone.csv',
]
ORIG_WEATHER_FILE = 'data/weather.csv'

# Set the target year and months (last 5 months ending previous month)
today = datetime.now()
end_month = (today.replace(day=1) - timedelta(days=1)).month
end_year = (today.replace(day=1) - timedelta(days=1)).year
start_month = end_month - 4 if end_month > 4 else 12 + (end_month - 4)
start_year = end_year if end_month > 4 else end_year - 1

# Generate new file names for 2025
def get_new_file_names(year, start_month):
    files = []
    for i in range(5):
        month = (start_month + i - 1) % 12 + 1
        y = year if start_month + i <= 12 else year + 1
        files.append(f"data/{y:04d}-{month:02d}_1H_zone.csv")
    return files

NEW_ZONE_FILES = get_new_file_names(end_year, start_month)
NEW_WEATHER_FILE = 'data/weather_2025.csv'

# --- Synthesize Zone Data ---
for orig, new, m in zip(ORIG_ZONE_FILES, NEW_ZONE_FILES, range(start_month, start_month+5)):
    df = pd.read_csv(orig)
    # Shift PUTime to new year/month, preserve hour/min/sec
    first_old = pd.to_datetime(df['PUTime'].iloc[0])
    new_year = end_year if m <= 12 else end_year + 1
    new_month = (m-1)%12+1
    day_offset = (datetime(new_year, new_month, 1) - datetime(first_old.year, first_old.month, 1)).days
    df['PUTime'] = pd.to_datetime(df['PUTime']) + pd.Timedelta(days=day_offset)
    df['PUTime'] = df['PUTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df.to_csv(new, index=False)
    print(f"Created {new}")

# --- Synthesize Weather Data ---
weather = pd.read_csv(ORIG_WEATHER_FILE)
weather_days = len(weather)
start_date = datetime(end_year if start_month <= 12 else end_year+1, start_month, 1)
weather['DATE'] = [(start_date + timedelta(days=i)).strftime('%m/%d/%y') for i in range(weather_days)]
weather.to_csv(NEW_WEATHER_FILE, index=False)
print(f"Created {NEW_WEATHER_FILE}")


Created data/2025-04_1H_zone.csv
Created data/2025-05_1H_zone.csv
Created data/2025-06_1H_zone.csv
Created data/2025-07_1H_zone.csv
Created data/2025-08_1H_zone.csv
Created data/weather_2025.csv


## Data Preprocessing for 2025 Data

We will preprocess the newly synthesized 2025 taxi demand and weather datasets using the same logic as in the original notebooks. This includes feature engineering, merging, and handling missing values. The resulting dataframe will be used for model training and evaluation.


In [ ]:
import glob
import numpy as np

# --- Load and Concatenate 2025 Zone Data ---
zone_files = sorted(glob.glob('data/2025-*_1H_zone.csv'))
df_list = [pd.read_csv(f) for f in zone_files]
df = pd.concat(df_list, ignore_index=True)

# --- Feature Engineering ---
df['PUTime'] = pd.to_datetime(df['PUTime'])
df['date'] = df['PUTime'].dt.date
df['hour'] = df['PUTime'].dt.hour
df['weekday'] = (df['PUTime'].dt.dayofweek < 5).astype(int)
df['peak_hour'] = ((df['hour'] >= 16) & (df['hour'] <= 20)).astype(int) | ((df['weekday'] == 1) & (df['hour'] >= 6) & (df['hour'] <= 10)).astype(int)

# --- Load and Merge Weather Data ---
weather = pd.read_csv('data/weather_2025.csv')
weather['DATE'] = pd.to_datetime(weather['DATE'], format='%m/%d/%y')
weather['date'] = weather['DATE'].dt.date

# Merge on date
merged = df.merge(weather, on='date', how='left')

# --- Add Lag Features (previous 24 hours and previous 30 days) ---
merged = merged.sort_values(['PUZone', 'PUTime'])
for lag in range(1, 25):
    merged[f'lag_{lag}'] = merged.groupby('PUZone')['Count'].shift(lag)
for day in range(1, 31):
    merged[f'lag_{day*24}'] = merged.groupby('PUZone')['Count'].shift(day*24)

# --- Handle Missing Values ---
merged = merged.fillna(0)

# --- Final Data ---
print('Shape after merge:', merged.shape)
display(merged.head())


Shape after merge: (264552, 29)


,Unnamed: 0,PUZone,Count,PUTime,date,hour,weekday,peak_hour,DATE,AWND,...,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT06,WT08
0,0,0,91,2025-04-01,2025-04-01,0,1,0,2025-04-01,5.59,...,310.0,15.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1,582,2025-04-01,2025-04-01,0,1,0,2025-04-01,5.59,...,310.0,15.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,2,292,2025-04-01,2025-04-01,0,1,0,2025-04-01,5.59,...,310.0,15.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,3,376,2025-04-01,2025-04-01,0,1,0,2025-04-01,5.59,...,310.0,15.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,4,67,2025-04-01,2025-04-01,0,1,0,2025-04-01,5.59,...,310.0,15.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Model Training and Comparison

We will now train and compare 7 advanced algorithms for taxi demand forecasting, including boosting and balancing techniques. The models will be evaluated using appropriate metrics, and the best-performing model will be highlighted.


In [13]:
pip install -r requirements.txt

  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl.metadata (15 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached catboost-1.2.5.tar.gz (69.6 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
 

  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [157 lines of output]
        Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
        Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
        Using cached jupyterlab-3.6.8-py3-none-any.whl.metadata (12 kB)
        Using cached conan-1.66.0.tar.gz (789 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'done'
        Using cached ipython-9.5.0-py3-none-any.whl.metadata (4.4 kB)
        Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
        Using cached tornado-6.5.2-cp39-abi3-win_amd64.whl.metadata

In [14]:
pip install catboost --pre

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install lightgbm==4.3.0

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install xgboost==2.1.1

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import catboost
import lightgbm
import xgboost
import sklearn
print("CatBoost:", catboost.__version__)
print("LightGBM:", lightgbm.__version__)
print("XGBoost:", xgboost.__version__)

CatBoost: 1.2.8
LightGBM: 4.3.0
XGBoost: 2.1.1


In [19]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install seaborn


Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install seaborn


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# --- Imports ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.compose import TransformedTargetRegressor

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings("ignore")

# --- Feature Engineering ---
def add_time_features(df):
    if 'PUTime' in df.columns:
        df['PUTime'] = pd.to_datetime(df['PUTime'])
        df['hour'] = df['PUTime'].dt.hour
        df['day_of_week'] = df['PUTime'].dt.dayofweek
        df['month'] = df['PUTime'].dt.month
    return df

def add_lag_features(df, target, lags=[1,2,3]):
    for lag in lags:
        df[f"{target}_lag_{lag}"] = df[target].shift(lag)
    df = df.fillna(method='bfill')
    return df

merged = add_time_features(merged)
merged = add_lag_features(merged, target='Count', lags=[1,2,3])

# --- Features & Target ---
target = "Count"
features = [col for col in merged.columns if col not in ["PUZone", "PUTime", "Count", "date", "DATE"]]

X = merged[features]
y = merged[target]

categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ]
)

# --- Objective for Optuna ---
def objective(trial):
    # Hyperparameters
    rf_params = {
        'n_estimators': trial.suggest_int('rf_n_estimators', 100, 500),
        'max_depth': trial.suggest_int('rf_max_depth', 10, 30),
        'min_samples_split': trial.suggest_int('rf_min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('rf_min_samples_leaf', 1, 5),
        'random_state': 42
    }
    xgb_params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 100, 500),
        'max_depth': trial.suggest_int('xgb_max_depth', 3, 10),
        'learning_rate': trial.suggest_float('xgb_lr', 0.01, 0.3, log=True),
        'objective': 'reg:squarederror',
        'random_state': 42
    }
    lgb_params = {
        'n_estimators': trial.suggest_int('lgb_n_estimators', 100, 500),
        'max_depth': trial.suggest_int('lgb_max_depth', 5, 20),
        'learning_rate': trial.suggest_float('lgb_lr', 0.01, 0.3, log=True),
        'random_state': 42
    }
    cat_params = {
        'n_estimators': trial.suggest_int('cat_n_estimators', 100, 500),
        'depth': trial.suggest_int('cat_depth', 4, 10),
        'learning_rate': trial.suggest_float('cat_lr', 0.01, 0.3, log=True),
        'verbose': 0,
        'random_state': 42
    }

    # --- Wrap regressors ---
    rf = RandomForestRegressor(**rf_params)
    xgb = TransformedTargetRegressor(regressor=XGBRegressor(**xgb_params))
    lgbm = TransformedTargetRegressor(regressor=LGBMRegressor(**lgb_params))
    cat = TransformedTargetRegressor(regressor=CatBoostRegressor(**cat_params))

    # --- Stacking ---
    stack_model = StackingRegressor(
        estimators=[('rf', rf), ('xgb', xgb), ('lgbm', lgbm), ('cat', cat)],
        final_estimator=GradientBoostingRegressor(n_estimators=200, random_state=42),
        n_jobs=-1
    )

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', stack_model)
    ])

    cv = RepeatedKFold(n_splits=5, n_repeats=1, random_state=42)
    r2 = cross_val_score(pipeline, X, y, cv=cv, scoring='r2', n_jobs=1)
    return r2.mean()

# --- Run Optuna ---
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best R²:", study.best_value)
print("Best Hyperparameters:", study.best_params)

[I 2025-09-09 15:41:49,050] A new study created in memory with name: no-name-13339e3e-49c0-414b-8280-6d2710e811ec
  0%|          | 0/50 [00:00<?, ?it/s]

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


  2%|▏         | 1/50 [1:56:30<95:09:08, 6990.78s/it]

[W 2025-09-09 17:38:19,817] Trial 0 failed with parameters: {'rf_n_estimators': 368, 'rf_max_depth': 18, 'rf_min_samples_split': 10, 'rf_min_samples_leaf': 1, 'xgb_n_estimators': 447, 'xgb_max_depth': 5, 'xgb_lr': 0.022651948347789407, 'lgb_n_estimators': 115, 'lgb_max_depth': 5, 'lgb_lr': 0.01424300965243176, 'cat_n_estimators': 465, 'cat_depth': 10, 'cat_lr': 0.027097389660999952} because of the following error: The value nan is not acceptable.
[W 2025-09-09 17:38:19,828] Trial 0 failed with value np.float64(nan).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [16]:
!pip install sparse

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 2.8/2.8 MB 15.7 MB/s  0:00:00
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
   -- ------------------------------------- 1.6/30.3 MB 8.5 MB/s eta 0:00:04
   --- ------------------------------------ 2.6/30.3 MB 6.5 MB/s eta 0:00:05
   ---- ----------------------------------- 3.1/30.3 MB 5.8 MB/s eta 0:00:05
   ---- ----------------------------------- 3.4/30.3 MB 4.7 MB/s eta 0:00:06
   ---- ----------------------------------- 3.7/30.3 MB 3.7 MB/s eta 0:00:08
   ----- ---------------------------------- 4.2/30.3 MB 3.2 MB/s eta 0:00:09
   ------ --------------------------------- 4.7/30.3 MB 3.1 MB/s eta 0:00:09
   ------ --------------------------------- 5.2/30.3 MB 3.1 MB/s eta 0:00:09
   ------- -------------------------------- 6.0/30.3 MB 3.1 MB/s eta 0:00:08
   --------- ------------------------------ 7.1/30.3 MB 3.3 MB/s eta 0:00:08
   ---------- ---

  You can safely remove it manually.
  You can safely remove it manually.


## Expanding to 11 Algorithms & Hybrid Models (Including LSTM)

We will now expand our comparison to 11 top algorithms, including deep learning (LSTM) and hybrid ensemble models. This will provide a comprehensive benchmark and highlight the best approaches for taxi demand forecasting in 2025.


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor, VotingRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

# --- Add more models ---
models.update({
    'LinearRegression': LinearRegression(),
    'SVR': SVR(),
})

# --- LSTM Preparation ---
lstm_features = features.copy()
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X[lstm_features])

def create_lstm_dataset(X, y, time_steps=24):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

X_lstm, y_lstm = create_lstm_dataset(X_scaled, y, time_steps=24)
X_lstm_train, X_lstm_val = X_lstm[:int(0.8*len(X_lstm))], X_lstm[int(0.8*len(X_lstm)):]
y_lstm_train, y_lstm_val = y_lstm[:int(0.8*len(y_lstm))], y_lstm[int(0.8*len(y_lstm)) :]

lstm_model = Sequential([
    LSTM(64, input_shape=(X_lstm_train.shape[1], X_lstm_train.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])
lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')

es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = lstm_model.fit(X_lstm_train, y_lstm_train, epochs=30, batch_size=128, validation_data=(X_lstm_val, y_lstm_val), callbacks=[es], verbose=1)
lstm_preds = lstm_model.predict(X_lstm_val).flatten()
lstm_mae = mean_absolute_error(y_lstm_val, lstm_preds)
lstm_rmse = mean_squared_error(y_lstm_val, lstm_preds, squared=False)
lstm_r2 = r2_score(y_lstm_val, lstm_preds)
results['LSTM'] = {'MAE': lstm_mae, 'RMSE': lstm_rmse, 'R2': lstm_r2}
print(f"LSTM: MAE={lstm_mae:.2f}, RMSE={lstm_rmse:.2f}, R2={lstm_r2:.2f}")

# --- Hybrid Models ---
# Stacking (using top 3 models)
top3 = list(results_df.index[:3])
stacking = StackingRegressor([
    (name, models[name]) for name in top3 if name in models
], final_estimator=LinearRegression())
stacking.fit(X_train, y_train)
stacking_preds = stacking.predict(X_val)
stacking_mae = mean_absolute_error(y_val, stacking_preds)
stacking_rmse = mean_squared_error(y_val, stacking_preds, squared=False)
stacking_r2 = r2_score(y_val, stacking_preds)
results['Stacking'] = {'MAE': stacking_mae, 'RMSE': stacking_rmse, 'R2': stacking_r2}
print(f"Stacking: MAE={stacking_mae:.2f}, RMSE={stacking_rmse:.2f}, R2={stacking_r2:.2f}")

# Voting Ensemble (all models)
voting = VotingRegressor([(name, model) for name, model in models.items()])
voting.fit(X_train, y_train)
voting_preds = voting.predict(X_val)
voting_mae = mean_absolute_error(y_val, voting_preds)
voting_rmse = mean_squared_error(y_val, voting_preds, squared=False)
voting_r2 = r2_score(y_val, voting_preds)
results['Voting'] = {'MAE': voting_mae, 'RMSE': voting_rmse, 'R2': voting_r2}
print(f"Voting: MAE={voting_mae:.2f}, RMSE={voting_rmse:.2f}, R2={voting_r2:.2f}")

# --- Update Results DataFrame and Plot ---
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('MAE')
display(results_df)
plt.figure(figsize=(12,6))
sns.barplot(x=results_df.index, y=results_df['MAE'])
plt.title('Model MAE Comparison (11+ Models)')
plt.ylabel('MAE')
plt.xticks(rotation=45)
plt.show()
